<a href="https://colab.research.google.com/github/AlessandraMayumi/python-generator/blob/main/python_generator_aws.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction

I want to share how to develop a implementation from the high level problem to a solution from a scratch code and increasing complexity

## Problem
Generate a response containing how many empty cells for each header in a csv file stored in a s3 bucket. This process have to be in a aws lambda.

# Hands on


### Create a project
I started this python study project in *GitHub* repository https://github.com/AlessandraMayumi/python-generator. Then I created this Colab nootebook and saved a copy of this notebook on github.

### Reading a csv file with pandas
Before thinking about aws resources, let's try to retrieve the result from a local csv file as expected in the Prom description.

#### References
- w3schools: https://www.w3schools.com/python/pandas/pandas_csv.asp



In [15]:
import pandas as pd

df = pd.read_csv('/content/sample_data/california_housing_test.csv')
print(df.head())

    id firstname  lastname                         email  \
0  397      Lita     Ardra        Lita.Ardra@yopmail.com   
1  398    Carree     Giule      Carree.Giule@yopmail.com   
2  399  Brandise   Goddard  Brandise.Goddard@yopmail.com   
3  400     Dotty   Eckblad     Dotty.Eckblad@yopmail.com   
4  401   Chickie  Gamaliel  Chickie.Gamaliel@yopmail.com   

                       email2 profession  
0        Lita.Ardra@gmail.com     doctor  
1      Carree.Giule@gmail.com     worker  
2  Brandise.Goddard@gmail.com     worker  
3     Dotty.Eckblad@gmail.com     worker  
4  Chickie.Gamaliel@gmail.com     worker  


### Prepare a test csv file
Generate a csv file and clear some cells

#### References
- python open - https://www.pythontutorial.net/python-basics/python-read-text-file/
- csv generator: https://extendsclass.com/csv-generator.html

In [71]:
import pandas as pd
import random

""" The test csv file should have some empty cells """

filename_test = 'mock_empty_register.csv'
filepath_test = f'/content/sample_data/{filename_test}'
filepath_original = '/content/sample_data/mock_register.csv'

def modify_empty_cells():
  df = pd.read_csv(filepath_original)

  num_row, num_col = df.shape

  for i in range(100):
    x = random.randrange(num_row)
    y = random.randrange(1, num_col)
    df.loc[x, df.columns[y]] = None

  df.to_csv(filepath_test)

  print(f'Test csv file generated: {filename_test}')


modify_empty_cells()


Test csv file generated: mock_empty_register.csv


## Working on the solution
After the test file is generated, let's work on the solution.

### Count empty cells
Pandas already have a easy way to do it.

### Convert dataframe into dict

In [88]:
df_test = pd.read_csv(filepath_test)

def get_empty_count(df):
  empty_count = df.isna().sum().to_dict()
  empty_count.pop('id')
  empty_count.pop('Unnamed: 0')
  print(f'Sucessfully count empty cells for each column: {empty_count}')
  return empty_count

response = get_empty_count(df_test)

Sucessfully count empty cells for each column: {'firstname': 19, 'lastname': 22, 'email': 18, 'email2': 17, 'profession': 24}


In [ ]:
# import cProfiler
# import stats

# profiler = cProfile.Profile()
# profiler.enable()

# f()

# profiler.disable()
# stats.print_state(())